# Reading mail body 

Importing packages ...

In [50]:
import pandas as pd
from bs4 import BeautifulSoup as BS
import re

Reading data ...

In [51]:
mails=pd.read_csv('email_message_excerpt.csv', encoding = "ISO-8859-1")

In [52]:
mails.dtypes

Id                                    object
ParentId                              object
ActivityId                            object
CreatedById                           object
CreatedDate                           object
LastModifiedDate                      object
LastModifiedById                      object
SystemModstamp                        object
TextBody                              object
HtmlBody                              object
Headers                               object
Subject                               object
Anon                                  object
from@test.com                         object
bcc@test.com                          object
CcAddress                             object
BccAddress                            object
Incoming                               int64
HasAttachment                          int64
Status                                 int64
MessageDate                           object
IsDeleted                              int64
MessageSiz

Let's only keep those columns which are of our interest

In [53]:
body=mails[['Id','TextBody','HtmlBody']]

In [63]:
sal=["from:","date:","to:","subject:","sent:","\xa0","\r",
    "\u200b","*x* external email *x*","url",
                              "sent from my iphone","please respond above this line.","prize update from:",'(concierge)',
    "(sr. concierge)"]

In [55]:
salutation = ["hi",
                                     "hi,",
                                     "dear",
                                     "hey",
                                     "hello",
                                     "good",'please respond above this line']

In [56]:
signature = ["warm regards",
                              "kind regards",
                              "regards",
                              "cheers",
                              "many thanks",
                              "sincerely",
                              "best,",
                              "talk soon",
                              "cordially",
                              "yours truly",
                              "thanking You",
                          "thanks,","thank you,","thank you!"]

### Reading body content from html body and writing it in HtmlSol column

In [65]:
for x in range(body.shape[0]):
    #print('#########################################################################################')
    trial=body['HtmlBody'][x]
    if pd.isnull(body['HtmlBody'][x]):
        t=sol='No mail'
        continue
    else:
        soup = BS(trial,'html.parser')
        for br in soup.find_all("br"):
            br.replace_with("\n")
        if soup.find_all('body'):
            try:
                if 'xmlns' in str(soup.find('html')):
                    t=soup.find('div').get_text()
                elif soup.find('div') is not None:
                    t=soup.find('div').find_all('div')[-1].get_text()
                else:
                    t=soup.find('body').get_text()
            except IndexError:
                t=soup.find('body').get_text()  
            mean=[]
            #t = t.replace('\r', '')
            t1=t.split('\n')
            flag=0
            for j in range(len(t1)):
                mean.append(t1[j])
                for k in signature:
                    if k in t1[j].lower():
                        flag=1
                        mean.remove(t1[j])
                        break
                if flag==1:
                    break
                if len(t1[j])>3:

                    #mean.append(t1[j])
                    for i in ( sal):
                        if i in t1[j].lower():

                            mean.remove(t1[j])
                            break
                else:
                    mean.remove(t1[j])
            if len(mean)>0:
                if mean[0].split()[0].lower() in salutation:
                    mean.remove(mean[0])
            if (len(mean)>5):
                sol=('\n'.join((mean[:3])))
            else:
                sol=('\n'.join((mean)))
        else:
            sol='No mail'
#     print(sol)
#     print('********')
#     print(t)
#     print(x)

    body.loc[body.index[x], 'HtmlSol'] = sol


    


/home/shreyas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Reading body content from text body and writing it in TextSol column

In [66]:
 for x in range((body.shape[0])):
    #print('##################################')
    sol=''
    trial=body['TextBody'][x]
    t = trial.replace('\r', '')
    mean=[]
    t1=t.split('\n')
    flag=0
    for j in range(len(t1)):
        mean.append(t1[j])
        if re.match('\d{2}/\d{2}/\d{4} \d{1}:\d{2} [A-Z]', t1[j]) or re.match('[A-Za-z]+ \d{2}, \d{4} \d{2}:\d{2} [A-Z]', t1[j]) or re.match('\d{2}/\d{2}/\d{4} \d{2}:\d{2} [A-Z]', t1[j]) or re.match('[A-Za-z]+ \d{2}, \d{4} \d{1}:\d{2} [A-Z]', t1[j]):
            mean.remove(t1[j])
        for k in signature:
            if k in t1[j].lower():
                flag=1
                mean.remove(t1[j])
                break
        if flag==1:
            break
        if len(t1[j])>3:

            #mean.append(t1[j])
            for i in ( sal):
                if i in t1[j].lower():
#                     print(t1[j])
#                     print(i)
                    mean.remove(t1[j])
                    break
        else:
            mean.remove(t1[j])
    if len(mean)>0:
        if mean[0].split()[0].lower() in salutation:
            mean.remove(mean[0])
    if (len(mean)>5):
        sol=('\n'.join((mean[:3])))
    else:
        sol=('\n'.join((mean)))
    body.loc[body.index[x], 'TextSol'] = sol


/home/shreyas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [71]:
body.head(5)

,Id,TextBody,HtmlBody,HtmlSol,TextSol
0,02s1Q00000UC1qdQAD,Prize update from: 0013600001hG1DQAA0\r\nPleas...,<html>\r\n \r\n <head>\r\n <meta ...,Just checking in here to make sure you don?t f...,Just checking in here to make sure you don?t f...
1,02s1Q00000UC1qeQAD,Prize update from: 0013600001ewt3wAAA\r\nPleas...,<html>\r\n \r\n <head>\r\n <meta ...,Since we've yet to hear from you regarding you...,Since we've yet to hear from you regarding you...
2,02s1Q00000UC1sWQAT,Hi\n\nSorry. November 17th is the day we picke...,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",Sorry. November 17th is the day we picked.\nAn...,Sorry. November 17th is the day we picked.\nAn...
3,02s1Q00000UC1w2QAD,"Hi Hannah, thanks for the suggestions, the Rho...","<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",,
4,02s1Q00000UC1x5QAD,This message was created automatically by the ...,NaN,NaN,This message was created automatically by the ...


### Writing the results dataframe into a csv file

In [69]:
body.to_csv('Result.csv')